# Power Quality Classification using CNN

This notebook focusses on developing a Convolutional Neural Network which classifies a particular power signal into its respective power quality condition. The dataset used here contains signals which belong to one of the 6 classes(power quality condition). This means that each signal is characterized by 256 data points. Here the signals provided are in time domain.

In [ ]:
#importing the required libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
from scipy.fft import fft,fftfreq
from scipy import signal
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
#loading the dataset using pandas
x_train = pd.read_csv("../input/power-quality-distribution-dataset-2/VoltageL1Train.csv")
y_train = pd.read_csv("../input/power-quality-distribution-dataset-2/outputTrain.csv")
x_test = pd.read_csv("../input/power-quality-distribution-dataset-2/VoltageL1Test.csv")
y_test = pd.read_csv("../input/power-quality-distribution-dataset-2/outputTest.csv")

In [ ]:
print("x_train",x_train.shape)
print("y_train",y_train.shape)
print("x_test",x_test.shape)
print("y_test",y_test.shape)

## Data Preprocessing

This segment of notebook contains all the preprocessing steps which are performed on the data.

In [ ]:
#dropna() function is used to remove all those rows which contains NA values
x_train.dropna(axis=0,inplace=True)
y_train.dropna(axis=0,inplace=True)
x_test.dropna(axis=0,inplace=True)
y_test.dropna(axis=0,inplace=True)

In [ ]:
#shape of the data frames after dropping the rows containing NA values
print("x_train",x_train.shape)
print("y_train",y_train.shape)
print("x_test",x_test.shape)
print("y_test",y_test.shape)

In [ ]:
#here we are constructing the array which will finally contain the column names
header =[]
for i in range(1,x_train.shape[1]+1):
    header.append("Col"+str(i))

In [ ]:
#assigning the column name array to the respectinve dataframes
x_train.columns = header
x_test.columns = header

In [ ]:
#assinging the column name for the y_train and y_test
header = ["output"]
y_train.columns = header
y_test.columns = header

In [ ]:
x_train.head()

In [ ]:
x_test.head()

In [ ]:
y_train.head()

In [ ]:
y_test.head()

In [ ]:
#further splitting the train dataset to train and validation
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

In [ ]:
print('x_train',x_train.shape)
print('y_train',y_train.shape)
print('x_val',x_val.shape)
print('y_val',y_val.shape)
print('x_test',x_test.shape)
print('y_test',y_test.shape)

In [ ]:
# get_dummies function is used here to perform one hot encoding of the y_* numpy arrays
y_train_hot = pd.get_dummies(y_train['output'])
y_test_hot = pd.get_dummies(y_test['output'])
y_val_hot = pd.get_dummies(y_val['output'])

In [ ]:
y_train_hot.head()

In [ ]:
y_train_arr = y_train_hot.to_numpy()
y_test_arr = y_test_hot.to_numpy()
y_val_arr = y_val_hot.to_numpy()
print("y_train:",y_train_arr.shape)
print("y_test:",y_test_arr.shape)
print("y_val:",y_val_arr.shape)
no_of_classes = y_train_arr.shape[1]

### Data transformation

The data transformation steps employed here are as follows:<br>

1) Fourier Transform<br>
2) Normalization

In [ ]:
x_train_tr = x_train.to_numpy()
x_test_tr = x_test.to_numpy()
x_val_tr = x_val.to_numpy()

In [ ]:
for i in range(0,x_train.shape[0]):
    x_train_tr[i][:] = np.abs(fft(x_train_tr[i][:]))
    
for i in range(0,x_test.shape[0]):
    x_test_tr[i][:] = np.abs(fft(x_test_tr[i][:]))

for i in range(0,x_val.shape[0]):
    x_val_tr[i][:] = np.abs(fft(x_val_tr[i][:]))

In [ ]:
transform = StandardScaler()
x_train_tr = transform.fit_transform(x_train)
x_test_tr = transform.fit_transform(x_test)
x_val_tr = transform.fit_transform(x_val)

In [ ]:
print("Training",x_train_tr.shape)
print(y_train_arr.shape)
print("Validation",x_val_tr.shape)
print(y_val_arr.shape)
print("Test",x_test_tr.shape)
print(y_test_arr.shape)
sampling_rate = x_train_tr.shape[1]

## Model creation and training

In [ ]:
#Reshaping the Data so that it could be used in 1D CNN
x_train_re = x_train_tr.reshape(x_train_tr.shape[0],x_train_tr.shape[1], 1)
x_test_re = x_test_tr.reshape(x_test_tr.shape[0],x_test_tr.shape[1], 1)
x_val_re = x_val_tr.reshape(x_val_tr.shape[0],x_val_tr.shape[1], 1)

In [ ]:
x_train_re.shape

In [ ]:
#importing required modules for working with CNN
import tensorflow as tf
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Convolution1D, ZeroPadding1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense
from tensorflow.keras.regularizers import l2

In [ ]:
#initializing required parameters for the model
batch_size = 64
num_classes = 6
epochs = 20
input_shape=(x_train_tr.shape[1], 1)

In [ ]:
model = Sequential()
model.add(Conv1D(128, kernel_size=3,padding = 'same',activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(128,kernel_size=3,padding = 'same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2)))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
#compiling the model

log_dir = "logs2/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
#training the model
history = model.fit(x_train_re, y_train_hot, batch_size=batch_size, epochs=epochs, validation_data=(x_val_re, y_val_hot), callbacks=[tensorboard_callback])


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs2/fit

In [ ]:
print(model.metrics_names)

## Model evaluation

In [ ]:
print("min val:",min(history.history['val_accuracy']))
print("avg val",np.mean(history.history['val_accuracy']) )
print("max val:",max(history.history['val_accuracy']))
print()
print("min train:",min(history.history['accuracy']))
print("avg train",np.mean(history.history['accuracy']) )
print("max train:",max(history.history['accuracy']))

In [ ]:
pred_acc = model.evaluate(x_test_re,y_test_hot)
print("Test accuracy is {}".format(pred_acc))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn

In [ ]:
array = confusion_matrix(y_test_hot.to_numpy().argmax(axis=1), model.predict(x_test_re).argmax(axis=1))

In [ ]:
array

In [ ]:
to_cm = pd.DataFrame(array, index = [i for i in ["Type-1","Type-2","Type-3","Type-4","Type-5","Type-6"]],
                  columns = [i for i in ["Type-1","Type-2","Type-3","Type-4","Type-5","Type-6"]])
plt.figure(figsize = (13,9))
sn.heatmap(to_cm, annot=True)